In [1]:
# !python3 -m pip uninstall tensorflow tensorflow-probability nsc -y
# !python3 -m pip install tensorflow tensorflow-probability -q

In [2]:
# !python3 -m pip uninstall nsc -y -q
# !python3 -m pip install -i https://test.pypi.org/simple/ nsc -q

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set()
from collections import defaultdict
import math
from typing import List

# From nsc lib
import nsc
from nsc import distributions as nsd
from nsc.util import function as nsc_func
from nsc.util.function import coupled_logarithm, coupled_exponential
# nsd = nsc.distributions

Importing NSC lib v0.0.2.1.


### 1) CoupledNormalDistribution function version

In [4]:
def norm_CG(sigma, kappa):
    if kappa == 0:
        result = math.sqrt(2*math.pi) * sigma
    elif kappa < 0:
        result = math.sqrt(math.pi) * sigma * math.gamma((-1+kappa) / (2*kappa)) / float(math.sqrt(-1*kappa) * math.gamma(1 - (1 / (2*kappa))))
    else:
        result = math.sqrt(math.pi) * sigma * math.gamma(1 / (2*kappa)) / float(math.sqrt(kappa) * math.gamma((1+kappa)/(2*kappa)))
  
    return result

In [5]:
def CoupledNormalDistribution(mean, sigma, kappa, alpha):
    """
    Short description
    
    Inputs
    ----------
    x : Input variable in which the coupled logarithm is applied to.
    mean : 
    sigma : 
    kappa : Coupling parameter which modifies the coupled logarithm function.
    dim : The dimension of x, or rank if x is a tensor. Not needed?
    """

    assert sigma > 0, "std must be greater than 0."
    assert alpha in [1, 2], "alpha must be set to either 1 or 2."

    
    if kappa >= 0:
        input = np.arange(mean-20, mean+20, (20+mean - -20+mean)/(2**16+1))
    else:
        x1 = mean - ((-1*sigma**2) / kappa)**0.5
        x2 = mean + ((-1*sigma**2) / kappa)**0.5
        input = np.arange(mean - ((-1*sigma**2) / kappa)**0.5, mean + ((-1*sigma**2) / kappa)**0.5, (x2-x1)/(2**16+1))
 
    normCGvalue = 1/float(norm_CG(sigma, kappa))
    
    coupledNormalDistributionResult = normCGvalue * (coupled_exponential((input - mean)**2/sigma**2, kappa)) ** -0.5
  
    return coupledNormalDistributionResult

In [6]:
kappa, alpha, dim = 0.5, 2, 1

In [7]:
mu, sigma = 0, 1 # mean and standard deviation
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
y = CoupledNormalDistribution(mu, sigma, kappa, alpha)

dx = np.arange(mu-20, mu+20, (20+mu - -20+mu)/(2**16+1))[1] - np.arange(mu-20, mu+20, (20+mu - -20+mu)/(2**16+1))[0]

### 2) CoupledNormalDistribution class

In [8]:
import numpy as np
import math
from typing import List


# From nsc lib
import nsc
from nsc import distributions as nsd
from nsc.util import function as nsc_func
from nsc.util.function import coupled_logarithm, coupled_exponential
# nsd = nsc.distributions

import ipdb

In [9]:
class CoupledNormal:
    """Coupled Normal Distribution.

    This distribution has parameters: location `loc`, 'scale', coupling `kappa`,
    and `alpha`.

    """
    def __init__(self,
                 loc: [int, float, List, np.ndarray],
                 scale: [int, float, List, np.ndarray],
                 kappa: [int, float] = 0.,
                 alpha: int = 2,
                 validate_args: bool = True
                 ):
        loc = np.asarray(loc) if isinstance(loc, List) else loc
        scale = np.asarray(scale) if isinstance(scale, List) else scale
        if validate_args:
            assert isinstance(loc, (int, float, np.ndarray)), "loc must be either an int/float type for scalar, or an list/ndarray type for multidimensional."
            assert isinstance(scale, (int, float, np.ndarray)), "scale must be either an int/float type for scalar, or an list/ndarray type for multidimensional."
            assert type(loc) == type(scale), "loc and scale must be the same type."
            if isinstance(loc, np.ndarray):
                # assert loc.shape == scale.shape, "loc and scale must have the same dimensions (check respective .shape())."
                assert np.all((scale >= 0)), "All scale values must be greater or equal to 0."            
            else:
                assert scale >= 0, "scale must be greater or equal to 0."            
            assert isinstance(kappa, (int, float)), "kappa must be an int or float type."
            assert isinstance(alpha, int), "alpha must be an int that equals to either 1 or 2."
            assert alpha in [1, 2], "alpha must be equal to either 1 or 2."
        self.loc = loc
        self.scale = scale
        self.kappa = kappa
        self.alpha = alpha
        self.dim = self._n_dim()

    def _n_dim(self):
        return 1 if self._event_shape() == [] else self._event_shape()[0]

    def _batch_shape(self) -> List:
        if self._rank(self.loc) == 0:
            # return [] signifying single batch of a single distribution
            return []
        else:
            # return the batch shape in list format
            return list(self.loc.shape)

    def _event_shape(self) -> List:
        # For univariate Coupled Normal distribution, event shape is always []
        # [] signifies single random variable dim (regardless of batch size)
        return []

    def _rank(self, value: [int, float, np.ndarray]) -> int:
        # specify the rank of a given value, with rank=0 for a scalar and rank=ndim for an ndarray
        if isinstance(value, (int, float)):
            return 0 
        else:
            return len(value.shape)

    def prob(self, X: [List, np.ndarray]) -> np.ndarray:
        # Check whether input X is valid
        X = np.asarray(X) if isinstance(X, List) else X
        assert isinstance(X, np.ndarray), "X must be a List or np.ndarray."
        # assert type(X[0]) == type(self.loc), "X samples must be the same type as loc and scale."
        if isinstance(X[0], np.ndarray):
            assert X[0].shape == self.loc.shape, "X samples must have the same dimensions as loc and scale (check respective .shape())."
        # Calculate PDF with input X
        X_norm = (X-self.loc)**2 / self.scale**2
        norm_term = self._normalized_term()
        p = (coupled_exponential(X_norm, self.kappa))**-0.5 / norm_term
        # normCGvalue =  1/float(norm_CG(scale, kappa))
        # coupledNormalDistributionResult = normCGvalue * (coupled_exponential(y, kappa)) ** -0.5
        return p

    # Normalization of 1-D Coupled Gaussian (NormCG)
    def _normalized_term(self) -> [int, float, np.ndarray]:
        if self.kappa == 0:
            norm_term = np.sqrt(2*np.pi) * self.scale
        elif self.kappa < 0:
            gamma_num = math.gamma(self.kappa-1) / (2*self.kappa)
            gamma_dem = math.gamma(1 - (1 / (2*self.kappa)))
            norm_term = (np.sqrt(np.pi)*self.scale*gamma_num) / float(np.sqrt(-1*self.kappa)*gamma_dem)
        else:
            gamma_num = math.gamma(1 / (2*self.kappa))
            gamma_dem = math.gamma((1+self.kappa)/(2*self.kappa))
            norm_term = (np.sqrt(np.pi)*self.scale*gamma_num) / float(np.sqrt(self.kappa)*gamma_dem)
        return norm_term

    def __repr__(self) -> str:
        return f"<nsc.distributions.{self.__class__.__name__} batch_shape={str(self._batch_shape())} event_shape={str(self._event_shape())}>"


In [41]:
class MultivariateCoupledNormal(CoupledNormal):
    """Multivariate Coupled Normal Distribution.

    This distribution has parameters: location `loc`, 'scale', coupling `kappa`,
    and `alpha`.

    """
    def __init__(self,
                 loc: [int, float, List, np.ndarray],
                 scale: [int, float, List, np.ndarray],
                 kappa: [int, float] = 0.,
                 alpha: int = 2,
                 validate_args: bool = True
                 ):
        if validate_args:
            assert isinstance(loc, (list, np.ndarray)), "loc must be either a list or ndarray type. Otherwise use CoupledNormal."
            assert isinstance(scale, (list, np.ndarray)), "scale must be either a list or ndarray type. Otherwise use CoupledNormal."
        super(MultivariateCoupledNormal, self).__init__(
            loc=loc,
            scale=scale,
            kappa=kappa,
            alpha=alpha,
            validate_args=validate_args
        )
        if self._rank(self.scale) == 1:
            self.scale = np.diag(self.scale)
        # Ensure that scale is indeed positive definite
        # assert self.is_positive_definite(self.scale), "scale must be positive definite."
        
    # Credit: https://stackoverflow.com/questions/16266720/find-out-if-matrix-is-positive-definite-with-numpy
    def is_positive_definite(self, A: np.ndarray) -> bool:
        if np.array_equal(A, A.T):
            try:
                np.linalg.cholesky(A)
                return True
            except np.linalg.LinAlgError:
                return False
        else:
            return False

    def _batch_shape(self) -> List:
        if self._rank(self.loc) == 1:
            # return [] signifying single batch of a single distribution
            return []
        else:
            # return [batch size]
            return list(self.loc.shape[:-1])

    def _event_shape(self) -> List:
        if self._rank(self.loc) == 1:
            # if loc is only a vector
            return list(self.loc.shape)
        else:
            # return [n of random variables] when rank >= 2
            return [self.loc.shape[-1]]

    def _rank(self, value: [int, float, np.ndarray]) -> int:
        # specify the rank of a given value, with rank=0 for a scalar and rank=ndim for an ndarray
        if isinstance(value, (int, float)):
            return 0 
        else:
            return len(value.shape)

    def prob(self, X: [List, np.ndarray]) -> np.ndarray:
        assert X.shape[-1] ==  self.loc.shape[-1], "input X and loc must have the same dims."
        sigma = np.matmul(self.scale, self.scale)
        sigma_inv = np.linalg.inv(sigma)
        _normalized_X = lambda x: np.linalg.multi_dot([x, sigma_inv, x])
        X_norm = np.apply_along_axis(_normalized_X, 1, X)
        norm_term = self._normalized_term()
        p = (coupled_exponential(X_norm, self.kappa))**(-1/self.alpha) / norm_term
        return p

    # Normalization of the multivariate Coupled Gaussian (NormMultiCoupled)
    def _normalized_term(self) -> [int, float, np.ndarray]:
        sigma = np.matmul(self.scale, self.scale.T)
        sigma_det = np.linalg.det(sigma)
        if self.alpha == 1:
            return sigma_det**0.5 / (1 + (-1 + self.dim)*self.kappa)
        else:  # self.alpha == 2
            gamma_num = math.gamma((1 + (-1 + self.dim)*self.kappa) / (2*self.kappa))
            gamma_dem = math.gamma((1 + self.dim*self.kappa) / (2*self.kappa))
            return (np.sqrt(np.pi) * sigma_det**0.5 * gamma_num) / (np.sqrt(self.kappa) * gamma_dem)


***Test***

In [ ]:
loc, scale, kappa, alpha = 0., 1., 0.5, 2
# X_input = np.arange(mu-20., mu+20., (20.+mu - -20.+mu)/(2.**16.+1.), dtype=float)
six_scale = 6.*scale
# X_input = np.arange(mu-6.*sigma, mu+6.*sigma, (20.+mu - -20.+mu)/(2.**16.+1.), dtype=float)
X_input = np.linspace(mu-six_scale, mu+six_scale, 1000)

In [ ]:
X_input.shape

Coupled Normal distribution

In [ ]:
cn = CoupledNormal(loc=mu, scale=sigma)

In [ ]:
cn

In [ ]:
print(cn.dim)
print(cn._normalized_term())
print(cn.prob(X_input))

Test for really low kappa value

In [16]:
kappa=0.003

In [17]:
cn = CoupledNormal(loc=mu, scale=sigma, kappa=kappa)

In [18]:
print(1 / (2*kappa))
math.gamma(100)

166.66666666666666


9.332621544394415e+155

In [19]:
print(cn.dim)
print(cn._normalized_term())
print(cn.prob(X_input))

1
2.508508948180413
[1.42896327e-08 1.52522011e-08 1.62778955e-08 1.73707354e-08
 1.85349883e-08 1.97751838e-08 2.10961296e-08 2.25029279e-08
 2.40009925e-08 2.55960675e-08 2.72942464e-08 2.91019925e-08
 3.10261610e-08 3.30740213e-08 3.52532813e-08 3.75721129e-08
 4.00391789e-08 4.26636611e-08 4.54552902e-08 4.84243777e-08
 5.15818489e-08 5.49392780e-08 5.85089250e-08 6.23037750e-08
 6.63375792e-08 7.06248983e-08 7.51811481e-08 8.00226476e-08
 8.51666701e-08 9.06314963e-08 9.64364708e-08 1.02602061e-07
 1.09149921e-07 1.16102954e-07 1.23485386e-07 1.31322836e-07
 1.39642391e-07 1.48472692e-07 1.57844013e-07 1.67788354e-07
 1.78339533e-07 1.89533287e-07 2.01407373e-07 2.14001678e-07
 2.27358336e-07 2.41521846e-07 2.56539199e-07 2.72460013e-07
 2.89336668e-07 3.07224461e-07 3.26181751e-07 3.46270130e-07
 3.67554585e-07 3.90103682e-07 4.13989749e-07 4.39289077e-07
 4.66082123e-07 4.94453727e-07 5.24493340e-07 5.56295261e-07
 5.89958884e-07 6.25588967e-07 6.63295895e-07 7.03195978e-07
 7.4

Coupled Normal multiple distributions with batch vector size of 2

In [20]:
cn = CoupledNormal(loc=[0., 1.], scale=[1., 2.])
cn

<nsc.distributions.CoupledNormal batch_shape=[2] event_shape=[]>

In [21]:
print(cn.dim)
print(cn._normalized_term())
# print(cn.prob(X_input))

1
[2.50662827 5.01325655]


Coupled Normal multiple distributions with batch matrix size of 3x2

In [22]:
cn = CoupledNormal(loc=[[0., 1.], [0., 1.], [0., 1.]], scale=[[1., 2.], [1., 2.], [1., 2.]])
cn

<nsc.distributions.CoupledNormal batch_shape=[3, 2] event_shape=[]>

**Multivariate Coupled Normal distribution when d=2**

In [42]:
# batch_shape is [] while event_shape is [2]
mcn = MultivariateCoupledNormal(loc=[0., 1.], scale=[1., 2.], kappa=0.1)
mcn

<nsc.distributions.MultivariateCoupledNormal batch_shape=[] event_shape=[2]>

In [43]:
print(mcn.dim)
print(mcn.loc)
print(mcn.scale)
print(mcn.kappa)
print(mcn.alpha)

2
[0. 1.]
[[1. 0.]
 [0. 2.]]
0.1
2


In [44]:
mcn._normalized_term()

4.889680162071525

Create 2nd X variable from -12 to 12.

In [45]:
twelve_scale = 12.*scale
# X_input = np.arange(mu-6.*sigma, mu+6.*sigma, (20.+mu - -20.+mu)/(2.**16.+1.), dtype=float)
X2_input = np.linspace(mu-twelve_scale, mu+twelve_scale, 1000)

In [46]:
X_2D = np.column_stack((X_input, X2_input))
X_2D.shape

(1000, 2)

In [47]:
X_2D

array([[ -6.        , -12.        ],
       [ -5.98798799, -11.97597598],
       [ -5.97597598, -11.95195195],
       ...,
       [  5.97597598,  11.95195195],
       [  5.98798799,  11.97597598],
       [  6.        ,  12.        ]])

Here is the dataset of X_2D, we can see that the sample size (m) is 1000 and the dim (d) for each sample is 2.

In [52]:
mcn.prob(X_2D)

array([1.92638620e-06, 1.96402683e-06, 2.00246169e-06, 2.04170873e-06,
       2.08178637e-06, 2.12271343e-06, 2.16450921e-06, 2.20719347e-06,
       2.25078645e-06, 2.29530888e-06, 2.34078200e-06, 2.38722755e-06,
       2.43466781e-06, 2.48312560e-06, 2.53262431e-06, 2.58318788e-06,
       2.63484086e-06, 2.68760838e-06, 2.74151620e-06, 2.79659071e-06,
       2.85285896e-06, 2.91034864e-06, 2.96908815e-06, 3.02910658e-06,
       3.09043374e-06, 3.15310020e-06, 3.21713725e-06, 3.28257698e-06,
       3.34945229e-06, 3.41779689e-06, 3.48764532e-06, 3.55903299e-06,
       3.63199621e-06, 3.70657218e-06, 3.78279905e-06, 3.86071593e-06,
       3.94036290e-06, 4.02178105e-06, 4.10501252e-06, 4.19010052e-06,
       4.27708933e-06, 4.36602438e-06, 4.45695222e-06, 4.54992062e-06,
       4.64497855e-06, 4.74217621e-06, 4.84156511e-06, 4.94319807e-06,
       5.04712924e-06, 5.15341418e-06, 5.26210988e-06, 5.37327477e-06,
       5.48696881e-06, 5.60325348e-06, 5.72219188e-06, 5.84384871e-06,
      

***Multivariate Coupled Normal distribution when d=8***

In [57]:
# batch_shape is [] while event_shape is [2]
mcn = MultivariateCoupledNormal(loc=[0., 1., 2., 3., 4., 5., 6., 7.],
                                scale=[1., 2., 3., 4., 5., 6., 7., 8.],
                                kappa=0.1
                               )
mcn

<nsc.distributions.MultivariateCoupledNormal batch_shape=[] event_shape=[8]>

In [61]:
print(mcn.dim)
print(mcn.loc)
print(mcn.scale)
print(mcn.kappa)
print(mcn.alpha)

8
[0. 1. 2. 3. 4. 5. 6. 7.]
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 4. 0. 0. 0. 0.]
 [0. 0. 0. 0. 5. 0. 0. 0.]
 [0. 0. 0. 0. 0. 6. 0. 0.]
 [0. 0. 0. 0. 0. 0. 7. 0.]
 [0. 0. 0. 0. 0. 0. 0. 8.]]
0.1
2


Create 6 more variables

In [58]:
two_scale = 2.*scale
four_scale = 4.*scale
eight_scale = 8.*scale
ten_scale = 10.*scale
fourteen_scale = 14.*scale
sixteen_scale = 16.*scale

X3_input = np.linspace(mu-two_scale, mu+two_scale, 1000)
X4_input = np.linspace(mu-four_scale, mu+four_scale, 1000)
X5_input = np.linspace(mu-eight_scale, mu+eight_scale, 1000)
X6_input = np.linspace(mu-ten_scale, mu+ten_scale, 1000)
X7_input = np.linspace(mu-fourteen_scale, mu+fourteen_scale, 1000)
X8_input = np.linspace(mu-sixteen_scale, mu+sixteen_scale, 1000)

In [60]:
X_8D = np.column_stack((X_input, X2_input, X3_input, X4_input, X5_input, X6_input, X7_input, X8_input))
X_8D.shape

(1000, 8)

Run prob

In [62]:
mcn.prob(X_8D)

array([4.81232826e-11, 4.90837687e-11, 5.00649993e-11, 5.10674558e-11,
       5.20916314e-11, 5.31380316e-11, 5.42071743e-11, 5.52995905e-11,
       5.64158242e-11, 5.75564333e-11, 5.87219892e-11, 5.99130781e-11,
       6.11303005e-11, 6.23742720e-11, 6.36456240e-11, 6.49450034e-11,
       6.62730736e-11, 6.76305149e-11, 6.90180245e-11, 7.04363174e-11,
       7.18861267e-11, 7.33682043e-11, 7.48833210e-11, 7.64322673e-11,
       7.80158539e-11, 7.96349122e-11, 8.12902950e-11, 8.29828767e-11,
       8.47135544e-11, 8.64832482e-11, 8.82929017e-11, 9.01434831e-11,
       9.20359854e-11, 9.39714274e-11, 9.59508544e-11, 9.79753385e-11,
       1.00045980e-10, 1.02163907e-10, 1.04330279e-10, 1.06546283e-10,
       1.08813140e-10, 1.11132100e-10, 1.13504448e-10, 1.15931501e-10,
       1.18414613e-10, 1.20955171e-10, 1.23554601e-10, 1.26214364e-10,
       1.28935962e-10, 1.31720936e-10, 1.34570868e-10, 1.37487381e-10,
       1.40472143e-10, 1.43526866e-10, 1.46653305e-10, 1.49853267e-10,
      

In [63]:
mcn.prob(X_8D).shape

(1000,)

***Multivariate Coupled Normal multiple distribution when d=2 and batch_size=3***

In [53]:
cn = MultivariateCoupledNormal(loc=[[0., 1.], [1., 2.], [2., 3.]], \
                               scale=[[0., 1.], [2., 3.], [4., 5.]],
                               kappa=0.1
                               )
cn

<nsc.distributions.MultivariateCoupledNormal batch_shape=[3] event_shape=[2]>

In [54]:
print(cn.dim)
print(cn.loc)
print(cn.scale)
print(cn.kappa)
print(cn.alpha)

2
[[0. 1.]
 [1. 2.]
 [2. 3.]]
[[0. 1.]
 [2. 3.]
 [4. 5.]]
0.1
2


In [55]:
cn = CoupledNormal(loc=[[0., 1., 0., 1.], [1., 2., 1., 2.], [2., 3., 2., 3.]], \
                   scale=[[0., 1., 0., 1.], [2., 3., 2., 3.], [4., 5., 4., 5.]]
                   )
cn.dim

1

In [56]:
print(cn.dim)
print(cn._normalized_term())

1
[[ 0.          2.50662827  0.          2.50662827]
 [ 5.01325655  7.51988482  5.01325655  7.51988482]
 [10.0265131  12.53314137 10.0265131  12.53314137]]


In [ ]:
# print(cn.prob(X_input))